In [1]:
from azureml.core import Workspace
from azureml.core import Experiment
from azureml.core import Dataset
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import ScriptRunConfig
from azureml.core import Model

from azureml.exceptions import ComputeTargetException

# own modules
import sys
sys.path.append('../')
from config.config_training import script_params_remote

Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception unknown locale: UTF-8.


In [2]:
# important parameters
compute_name = 'dp-100-v2'
project_folder = '../'

# 1. Configure Workspace

In [3]:
ws = Workspace.from_config('../config/config.json')

# 2. Configure Compute Target

In [4]:
try:
    comput_target = ComputeTarget(workspace=ws, name= compute_name)
    print('Comput target exists!')
except:
    config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_VS2'
                                                          ,vm_priority='lowpriority'
                                                          ,min_nodes=0
                                                          , max_nodes=1)
    comput_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)                                       

Comput target exists!


# 3. Configurate Dependencies

In [4]:
env = Environment('user-managed-env')
env.python.user_managed_dependencies = True

In [10]:
!pip list

Package                            Version    
---------------------------------- -----------
absl-py                            0.7.1      
alabaster                          0.7.10     
anaconda-client                    1.6.14     
anaconda-navigator                 1.9.6      
anaconda-project                   0.8.2      
appnope                            0.1.0      
appscript                          1.0.1      
asn1crypto                         0.24.0     
astor                              0.7.1      
astroid                            1.6.3      
astropy                            3.0.2      
attrs                              18.1.0     
autopep8                           1.4.4      
Babel                              2.5.3      
backcall                           0.1.0      
backports.shutil-get-terminal-size 1.0.0      
beautifulsoup4                     4.6.0      
bitarray                           0.8.1      
bkcharts                           0.2        
blaze        

# 5. Configure Experiment

In [5]:
src = ScriptRunConfig(source_directory=project_folder
                     ,script='./src/train_local.py'
                     ,arguments=script_params_remote)

In [6]:
src.run_config.target = 'local'
src.run_config.environment = env

In [7]:
experiment = Experiment(workspace=ws, name='test_v5')

# 6. Run Experiment und Monitor Training

In [8]:
run = experiment.submit(src)

# 7. Add trained motel to workspace

In [12]:
model = Model.register(workspace=ws
                       ,model_path='../trained_models/diabetes_model.pkl'
                       ,model_name='Diabetes_Classifier'
                       ,description='First trained model with Azure ML'
                       ,tags={'data-format': 'csv'}
                       ,properties={'Accuracy': run.get_metrics()['Accuracy']}
                       ,model_framework=Model.Framework.SCIKITLEARN  # Framework used to create the model.
                       ,model_framework_version='0.23.2'
                       ,datasets=[('training_data', tab_ds)]
                       )

KeyError: 'Accuracy'